In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from tqdm import tqdm

## 데이터 불러오기

In [2]:
raw_data=pd.read_csv('./data09/funda_train.csv')
data=raw_data.copy()
data.info()
#data.describe()  
# 날짜 헷갈리니까 합친다.
# 환불 금액이 보임. 환불을 먼저 없앤다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   store_id          int64  
 1   card_id           int64  
 2   card_company      object 
 3   transacted_date   object 
 4   transacted_time   object 
 5   installment_term  int64  
 6   region            object 
 7   type_of_business  object 
 8   amount            float64
dtypes: float64(1), int64(3), object(5)
memory usage: 450.2+ MB


## 환불금액제거.

In [ ]:
#합치기
data['date']=data['transacted_date']+' '+data['transacted_time']
data['date']=pd.to_datetime(data['date'])
#환불 제거
data_neg=data[data['amount']<0]
data_pos=data[data['amount']>0]
remove_data=pd.DataFrame()
# store_id 안나누고 하니까 시간이 너무 오래걸림. 
for i in tqdm(data.store_id.unique()):
    divided_data=data_pos[data_pos['store_id']==i] 
    divided_data2=data_neg[data_neg['store_id']==i] 
    #행 마다 조사.    
    for neg in divided_data2.to_records():  
        refund_store=neg['store_id']
        refund_id=neg['card_id'] 
        refund_date=neg['date']
        refund_amount=abs(neg['amount'])
        #행뽑기
        row=divided_data[divided_data['date']<=refund_date]
        row=row[row['card_id']==refund_id]
        row=row[row['amount']==refund_amount]
        
        if(len(row)!=0):
            refund_date=max(row['date']) 
            remove=divided_data[divided_data['date']==refund_date] 
            remove=remove[remove['card_id']==refund_id] 
            remove=remove[remove['amount']==refund_amount] 
            divided_data=divided_data.drop(index=remove.index) 

    remove_data=pd.concat([remove_data,divided_data],axis=0)

remove_data

#remove_data.to_csv('./data09/funda_remove_data.csv')

In [3]:
data=pd.read_csv('./data09/funda_remove_data.csv',index_col=0)
data['date_slice']=pd.to_datetime(data['date'].str.slice(stop=10))
data=data.set_index('date_slice')
#data.head(3)

C:\Users\ckdck\miniconda3\envs\tensor\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
data_month=pd.DataFrame()
for i in tqdm(data.store_id.unique()):
    data_num=data[data.store_id==i]
    sum_amount=data_num['amount'].resample(rule='m').sum()
    data_mon=pd.concat([sum_amount],axis=1)
    data_mon.insert(0,'store_id',i)
    data_month=pd.concat([data_month,data_mon],axis=0)
    
#data_month.head(10)

100%|██████████████████████████████████████████████████████████████████████████████| 1967/1967 [00:48<00:00, 40.19it/s]


## 지수 이동 평균

In [5]:
def sub(df,n):
    concat_3mon = pd.DataFrame(index=pd.to_datetime(['2019-03-31', '2019-04-30', '2019-05-31']))
    df_ema=pd.DataFrame(columns=['store_id','amount'])
    ema_sub = np.array([])

    for i in df.store_id.unique():
        df_set = pd.DataFrame(df[df.store_id == i].amount)
        wma_train = pd.concat([df_set, concat_3mon], axis=0)
        num_sub = wma_train.amount.ewm(span=n).mean()['2019-03':].sum()
        df_ema.loc[i,'store_id']=i
        df_ema.loc[i,'amount']=num_sub
        
    return df_ema

In [ ]:
for i in tqdm(range(4, 7)):
    ema_sub = sub(data_month, i)
    ema_sub.to_csv(f'./data09/funda_{i}ema_sub.csv',index=False)